In [7]:
from typing import Callable
from itertools import product

import pandas as pd

In [8]:
def build_table(columns: list[str] = ["P", "Q"]) -> pd.DataFrame:
    bools = [True, False]
    combinations = list(product(bools, repeat=len(columns)))
    df = pd.DataFrame(combinations, columns=columns)
    return df


def add_statement(
    df: pd.DataFrame, prop: Callable[..., bool], prop_name: str = "prop"
) -> pd.DataFrame:
    df[prop_name] = df.apply(lambda row: prop(*(row[c] for c in df.columns[:prop.__code__.co_argcount])), axis=1)
    return df

## Law of Commutativity

•	AND: p and q ≡ q and p  
•	OR: p or q ≡ q or p

In [9]:
df = build_table()

df = add_statement(df, lambda p, q: p and q, "P & Q")
df = add_statement(df, lambda p, q: q and p, "Q & P")

df = add_statement(df, lambda p, q: p or q, "P | Q")
df = add_statement(df, lambda p, q: q or p, "Q | P")

assert df["P & Q"].equals(df["Q & P"])
assert df["P | Q"].equals(df["Q | P"])

df

,P,Q,P & Q,Q & P,P | Q,Q | P
0,True,True,True,True,True,True
1,True,False,False,False,True,True
2,False,True,False,False,True,True
3,False,False,False,False,False,False


## Law of Associativity

* AND: p and (q and r) ≡ (p and q) and r  
* OR: p or (q or r) ≡ (p or q) or r

In [10]:
df = build_table(columns=["P", "Q", "R"])

df = add_statement(df, lambda p, q, r: p and (q and r), "P & (Q & R)")

df = add_statement(df, lambda p, q, r: (p and q) and r, "(P & Q) & R")

assert df["P & (Q & R)"].equals(df["(P & Q) & R"])

df = add_statement(df, lambda p, q, r: p or (q or r), "P | (Q | R)")

df = add_statement(df, lambda p, q, r: (p or q) or r, "(P | Q) | R")

assert df["P | (Q | R)"].equals(df["(P | Q) | R"])

df

,P,Q,R,P & (Q & R),(P & Q) & R,P | (Q | R),(P | Q) | R
0,True,True,True,True,True,True,True
1,True,True,False,False,False,True,True
2,True,False,True,False,False,True,True
3,True,False,False,False,False,True,True
4,False,True,True,False,False,True,True
5,False,True,False,False,False,True,True
6,False,False,True,False,False,True,True
7,False,False,False,False,False,False,False


## Law of Distributivity

•	p and (q or r) ≡ (p and q) or (p and r)  
•	p or (q and r) ≡ (p or q) and (p or r)

In [11]:
df = build_table(columns=["P", "Q", "R"])

df = add_statement(df, lambda p, q, r: p and (q or r), "P & (P | R)")

df = add_statement(df, lambda p, q, r: (p and q) or (p and r), "(P & Q) | (P & R)")

assert df.iloc[:,-1].equals(df.iloc[:,-2])

df = add_statement(df, lambda p, q, r: p or (q and r), "P | (P & R)")

df = add_statement(df, lambda p, q, r: (p or q) and (p or r), "(P | Q) & (P | R)")

assert df.iloc[:,-1].equals(df.iloc[:,-2])

df

,P,Q,R,P & (P | R),(P & Q) | (P & R),P | (P & R),(P | Q) & (P | R)
0,True,True,True,True,True,True,True
1,True,True,False,True,True,True,True
2,True,False,True,True,True,True,True
3,True,False,False,False,False,True,True
4,False,True,True,False,False,True,True
5,False,True,False,False,False,False,False
6,False,False,True,False,False,False,False
7,False,False,False,False,False,False,False


## Law of Identity

•	p and True ≡ p  
•	p or False ≡ p

In [12]:
df = build_table(columns=["P"])

df = add_statement(df, lambda p: p and True, "P & True")
df = add_statement(df, lambda p: p or False, "P | False")

assert df.P.equals(df.iloc[:, -2])
assert df.P.equals(df.iloc[:, -1])

df

,P,P & True,P | False
0,True,True,True
1,False,False,False
